# Product Segmentation

Here we will use two techniques, which are, KMeans clustering and Deep learning based classification so as to segment and classify different types of bottles coming out of the same manufacturing belt/line and allow them to be seperated by the mechanism present in the line, for the purpose of making batches or processing them differently.

### Imports

In [1]:
# Performing imports
import numpy as np
import cv2
import matplotlib.pyplot as plt
import shutil
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import glob
import warnings

### Defining Some File info

In [2]:
img_file = 'D:/My Work/Mechatronics/8th Sem. PPTs and Notes/Models/Product Segmentation/KNN Images/'
out_file = 'D:/My Work/Mechatronics/8th Sem. PPTs and Notes/Models/Product Segmentation/output/'
buf_file = 'D:/My Work/Mechatronics/8th Sem. PPTs and Notes/Models/Product Segmentation/buffer/'

In [3]:
# Deleting old test images
def get_ready():
    for img in os.listdir(img_file):
        path = os.path.join(img_file, img)
        os.remove(path)
    shutil.rmtree(out_file)
    os.mkdir('output')
    shutil.rmtree(buf_file)
    os.mkdir('buffer')
    return img_file

### Segment Image

In [4]:
def segment(img,k):
    # Change color to RGB (from BGR)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Reshaping the image into a 2D array of pixels and 3 color values (RGB)
    pixel_vals = image.reshape((-1,3))
    # Convert to float type
    pixel_vals = np.float32(pixel_vals)
    #the below line of code defines the criteria for the algorithm to stop running,
    #which will happen is 100 iterations are run or the epsilon (which is the required accuracy)
    #becomes 85%
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)
 
    # then perform k-means clustering with number of clusters defined as 3
    #also random centres are initially choosed for k-means clustering
    
    retval, labels, centers = cv2.kmeans(pixel_vals, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
 
    # convert data into 8-bit values
    centers = np.uint8(centers)
    segmented_data = centers[labels.flatten()]
 
    # reshape data into the original image dimensions
    segmented_image = segmented_data.reshape((image.shape))
    
    print('Image Segmented')
    
    cv2.imwrite(buf_file + 'img.jpg', segmented_image)

### Extracting each bottle from test image via yoloV3

In [5]:
def get_clusters(img,K):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # reading the pretrained yoloV3 files
    img = cv2.resize(img, None, fx=1, fy=1)
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    # reading and storing the classes on which yolo is trained on
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    #get layers of the network
    layer_names = net.getLayerNames()
    #Determine the output layer names from the YOLO model 
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    print("YOLO LOADED")
    height, width, channels = img.shape

    # USing blob function of opencv to preprocess image
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    #Detecting objects
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Finding and detecting objects and assigning the bounding boxes
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.65:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    #We use NMS function in opencv to perform Non-maximum Suppression
    #we give it score threshold and nms threshold as arguments.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    
    # Now storing the images separately
    count=0
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            img_store = img[y:y+h,x:x+w]
            if img_store.any() and label=='bottle':
                cv2.imwrite(img_file + 'img-' + str(count) + '.jpg', img_store)
                count+=1
                
    # we will feed all the images to the KMeans classifier
    preds = Kmeans(K)
    count=0
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (255,255,255)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            if img[y:y+h,x:x+w].any() and label=='bottle':
                cv2.putText(img, label + 'Cluster:' + str(preds[count]), (x, y+20),cv2.FONT_HERSHEY_SIMPLEX,0.75, color, 2)
                count+=1
    cv2.imshow("Image",img)
    cv2.imwrite("D:/My Work/Mechatronics/8th Sem. PPTs and Notes/Models/Product Segmentation/final/img.jpg", img)
    cv2.waitKey(0)

### Doing feature extraction from each individual image and KMeans clustering

In [6]:
def Kmeans(k=5):
    glob_dir = img_file + '/*.jpg'
    images = [cv2.resize(cv2.imread(file), (224, 224)) for file in glob.glob(glob_dir)]
    paths = [file for file in glob.glob(glob_dir)]
    images = np.array(np.float32(images).reshape(len(images), -1)/255)
    model = tf.keras.applications.MobileNetV2(include_top=False,
    weights='imagenet', input_shape=(224, 224, 3))
    predictions = model.predict(images.reshape(-1, 224, 224, 3))
    print('features extracted via mobilenet')
    pred_images = predictions.reshape(images.shape[0], -1)
    kmodel = KMeans(n_clusters = k,init = 'random',n_init = 'auto',max_iter=1000, tol = 0.001, random_state=42)
    kmodel.fit(pred_images)
    kpredictions = kmodel.predict(pred_images)
    shutil.rmtree('output')
    print('kmeans predictions done')
    for i in range(k):
        os.makedirs('output\cluster' + str(i))
    for i in range(len(paths)):
        shutil.copy2(paths[i], 'output\cluster'+str(kpredictions[i]))
    return kpredictions

### Main Function

In [7]:
def realtime_img(img,K):
    get_ready()
    #segment(img,3)
    #img = cv2.imread('buffer/img.jpg')
    get_clusters(img,K)

In [8]:
warnings.filterwarnings(action='ignore')
img = cv2.imread("Test Images/img-4.jpg")
realtime_img(img,2)

YOLO LOADED
1/1 [==============================] - 1s 830ms/step
features extracted via mobilenet
kmeans predictions done


### Real-time Implementation

In [9]:
def realtime_vid(K):
    get_ready()
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    if True:
        # set CUDA as the preferable backend and target
        print("[INFO] setting preferable backend and target to CUDA...")
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    #save all the names in file o the list classes
    classes = []
    kpredictions=[]
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    #get layers of the network
    layer_names = net.getLayerNames()
    #Determine the output layer names from the YOLO model 
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    video_capture = cv2.VideoCapture(0)
    while True:
        get_ready()
        # Capture frame-by-frame
        re,img = video_capture.read()
        if img.any():
            img = cv2.resize(img, None, fx=1.64, fy=1)
            height, width, channels = img.shape

            # USing blob function of opencv to preprocess image
            blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),swapRB=True, crop=False)
            #Detecting objects
            net.setInput(blob)
            outs = net.forward(output_layers)

            # Showing informations on the screen
            class_ids = []
            confidences = []
            boxes = []
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.6:
                        # Object detected
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)

                        # Rectangle coordinates
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

            #We use NMS function in opencv to perform Non-maximum Suppression
            #we give it score threshold and nms threshold as arguments.
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            font = cv2.FONT_HERSHEY_PLAIN
            colors = np.random.uniform(0, 255, size=(len(classes), 3))

            # Now storing the images separately
            count=0
            for i in range(len(boxes)):
                if i in indexes:
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]])
                    color = colors[class_ids[i]]
                    img_store = img[y:y+h,x:x+w]
                    if img_store.any() and label=='bottle':
                        cv2.imwrite(img_file + 'img' + str(count) + '.jpg', img_store)
                        count+=1

            # Kmeans
            glob_dir = img_file + '/*.jpg'
            images = [cv2.resize(cv2.imread(file), (224, 224)) for file in glob.glob(glob_dir)]
            paths = [file for file in glob.glob(glob_dir)]
            if len(images)!=0 :   
                pred_images = np.array(np.float32(images).reshape(len(images), -1)/255)
                #model = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
                #predictions = model.predict(images.reshape(-1, 224, 224, 3))
                #pred_images = images.reshape(-1, 224, 224, 3)
                #pred_images = predictions.reshape(images.shape[0], -1)
                kmodel = KMeans(n_clusters = K,init = 'random',n_init = 'auto',max_iter=1000, tol = 0.001, random_state=42)
                print(count)
                if kmodel.n_clusters<=count:
                    kmodel.fit(pred_images)
                    kpredictions = kmodel.predict(pred_images)

            count = 0

            for i in range(len(boxes)):
                if i in indexes:
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]])
                    color = colors[class_ids[i]]
                    cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                    if img[y:y+h,x:x+w].any()  and label=='bottle' and len(kpredictions)>0:
                        cv2.putText(img, label + 'Cluster:' + str(kpredictions[count]), (x, y+20),cv2.FONT_HERSHEY_SIMPLEX,0.75, color, 2)
                        count+=1
            cv2.imshow("Image",cv2.resize(img, (800,600)))
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    video_capture.release()
    cv2.destroyAllWindows()

In [13]:
realtime_vid(2)

[INFO] setting preferable backend and target to CUDA...
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
1
1
1
2
2
2
2
2
2
2
2
2
2
1
2
1
2
2
2
2
2
2
2
2
2
